# Transformer-based Language Model in Pytorch

Remarque : fait à l'arrache dans le notebook pour l'instant, déplacement dans un .py plus tard

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
import sys
import torchtext
sys.path.append("..")
#from common import get_positional_encodings
import nltk

In [2]:
#entrée : positional encoding + embeddings
#ajouter layer norm

In [163]:
# Paramètres

nb_decoders = 6
nb_heads = 8
vector_size = 512
vocab_size = 1000
head_size = vector_size//nb_heads
ffn_hidden_size = 100
seq_length = 15
attention_features_size = 5


In [20]:
class Transformer(nn.Module):
    def __init__(self, decoder):
        super(Transformer, self).__init__()
        self.decoder = decoder
        self.embedding = nn.Embedding(vocab_size, 4)
        self.finalfc = nn.Linear(vector_size, vocab_size)

    def forward(self, x):
        
        embedded = embedding(x)
        pos_encodings = torch.tensor(get_positional_encodings(seq_length, vector_size))
        x = embedded + pos_encodings
        
        for x in range(nb_decoders):
            x = self.decoder(x)
            
        x = F.softmax(self.finalfc(x))
        return x

In [12]:
class Decoder(nn.Module):
    "Decoder block applying multihead attention mechanism followed by a feedforward network"
    def __init__(self, multihead_attention, feedforward_network):
        super(Decoder, self).__init__()
        self.multihead_attention = multihead_attention
        self.feedforward_network = feedforward_network
        self.layernorm = nn.modules.normalization.LayerNorm

    def forward(self, x):
        #normalisation a la fin ou au debut ?
        mha = self.multihead_attention(self.layernorm(x))
        x += mha
        ffo = self.feedforward_network(self.layernorm(x))
        x += ffo
        return x
    

In [23]:
class FeedforwardNetwork(nn.Module):
    def __init__(self, vector_size, hidden_size):
        "Classic Feedforward Network with two layers"
        super(FeedforwardNetwork, self).__init__()
        self.hidden_size  = hidden_size
        self.fc1 = nn.Linear(vector_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, vector_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
ffn = FeedforwardNetwork(10,5)

Tests pour compréhension opérations torch, à virer

In [79]:
a = torch.tensor([[1,2,3],[3,4,5]])
b = torch.tensor([[1,2,1],[1,1,1],[3,3,3]])

In [84]:
a = torch.tensor([[[[1,2],[3,4]]],[[[1,2],[3,4]]]])
b = torch.tensor([[[[1,1],[10,20]],[[1,1],[10,20]]],[[[1,1],[10,20]],[[1,1],[10,20]]]])

In [117]:
b.transpose(-1,-2)

tensor([[[[ 1, 10],
          [ 1, 20]],

         [[ 1, 10],
          [ 1, 20]]],


        [[[ 1, 10],
          [ 1, 20]],

         [[ 1, 10],
          [ 1, 20]]]])

In [86]:
torch.matmul(a,b)

tensor([[[[21, 41],
          [43, 83]],

         [[21, 41],
          [43, 83]]],


        [[[21, 41],
          [43, 83]],

         [[21, 41],
          [43, 83]]]])

In [99]:
b.float()

tensor([[[[ 1.,  1.],
          [10., 20.]],

         [[ 1.,  1.],
          [10., 20.]]],


        [[[ 1.,  1.],
          [10., 20.]],

         [[ 1.,  1.],
          [10., 20.]]]])

In [118]:
torch.softmax(b.float(), dim=-1)

tensor([[[[5.0000e-01, 5.0000e-01],
          [4.5398e-05, 9.9995e-01]],

         [[5.0000e-01, 5.0000e-01],
          [4.5398e-05, 9.9995e-01]]],


        [[[5.0000e-01, 5.0000e-01],
          [4.5398e-05, 9.9995e-01]],

         [[5.0000e-01, 5.0000e-01],
          [4.5398e-05, 9.9995e-01]]]])

In [107]:
b.resize(2*2*2,2)

tensor([[ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20]])

In [115]:
b.view(-1,2)

tensor([[ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20]])

In [116]:
b.shape[-1]

2

In [123]:
torch.randn(5,3)

tensor([[-0.6129, -1.3946, -1.7488],
        [ 1.0109,  1.1781,  1.2424],
        [-1.1034,  2.5773, -0.4679],
        [ 0.4369, -0.4911, -0.2773],
        [ 1.0198,  0.2498, -2.3137]])

In [130]:
torch.randn(20).view(2,2,5)

tensor([[[-0.6981, -1.8573, -0.2192, -0.8049,  0.4418],
         [-0.0916, -0.2362, -0.5362,  0.8284, -0.7785]],

        [[ 1.4417,  3.0020,  0.0091,  1.0252, -0.0826],
         [-0.5025, -1.5500, -1.4086, -0.5481,  0.6857]]])

In [194]:
lineartest = nn.Linear(6,8)
a = lineartest(torch.randn(2,6))

In [195]:
a

tensor([[-0.0635,  0.0135, -0.1823, -0.6612, -0.0111,  0.1565, -0.4600,  0.1574],
        [-0.0561,  0.7180,  0.8235,  0.3773, -0.6609, -0.1302, -0.1962, -0.2220]],
       grad_fn=<AddmmBackward>)

In [151]:
a.reshape(5,2,4).transpose(1,0)

tensor([[[ 0.1325,  0.8591,  0.0375,  0.3665],
         [-0.0203,  0.5444,  0.4884,  0.3979],
         [ 0.2096,  0.4511,  0.4392, -0.1969],
         [-0.1197,  0.7166,  0.3569,  0.8327],
         [ 0.0605,  0.4559,  0.5451,  0.0421]],

        [[-0.4523,  0.3027, -0.2264,  0.4526],
         [-1.0083, -0.0689,  0.8458, -0.8348],
         [-1.2110,  0.1211, -0.0691,  0.1079],
         [-0.6316, -0.0602,  1.1216, -1.0492],
         [-1.2694, -0.0477,  0.5144, -0.5315]]], grad_fn=<TransposeBackward0>)

In [199]:
a = torch.randn(2,5,4)

In [200]:
a

tensor([[[ 0.4642, -0.1267, -0.1622,  1.5511],
         [-1.0926,  2.5877,  0.6728, -0.8214],
         [ 1.4204, -0.4553,  0.2679,  0.1943],
         [-0.4229,  1.9756,  0.6397,  0.0933],
         [ 1.5011, -1.7496, -0.6913, -0.3219]],

        [[-0.1436, -2.0942, -0.5142,  0.4932],
         [ 1.1384,  0.1072, -0.0375,  0.0156],
         [-0.7275,  0.6286,  0.5912, -0.6889],
         [ 2.0175, -1.1394,  0.6849, -0.0475],
         [-0.3738, -0.2450,  0.7022, -0.0796]]])

In [218]:
a.transpose(1,0).reshape(5,8)

tensor([[ 0.4642, -0.1267, -0.1622,  1.5511, -0.1436, -2.0942, -0.5142,  0.4932],
        [-1.0926,  2.5877,  0.6728, -0.8214,  1.1384,  0.1072, -0.0375,  0.0156],
        [ 1.4204, -0.4553,  0.2679,  0.1943, -0.7275,  0.6286,  0.5912, -0.6889],
        [-0.4229,  1.9756,  0.6397,  0.0933,  2.0175, -1.1394,  0.6849, -0.0475],
        [ 1.5011, -1.7496, -0.6913, -0.3219, -0.3738, -0.2450,  0.7022, -0.0796]])

In [222]:
#Mask ?
#Adapter pour batch gestion !

class MultiHeadAttention(nn.Module):
    "MultiHead Attention Block"
    def __init__(self, nb_heads, head_size, vector_size):
        super(MultiHeadAttention, self).__init__()
        self.nb_heads = nb_heads
        self.head_size = head_size
        self.vector_size = vector_size
        #bias = False ? (pour équivalence stricte avec une multiplication matricielle)
        self.w_q = nn.Linear(vector_size, vector_size)
        self.w_k = nn.Linear(vector_size, vector_size)
        self.w_v = nn.Linear(vector_size, vector_size)
        self.w_0 = nn.Linear(vector_size, vector_size)
        
    def mask_w(w):
        #
        return w
    
    def reshape_w(self, w):
        w = w.reshape(-1, self.nb_heads, self.head_size)
        w = w.transpose(1,0)
        return w
        
    def forward(self, x): 
        q = self.reshape_w(self.w_q(x))
        k = self.reshape_w(self.w_k(x))
        v = self.reshape_w(self.w_v(x))
        
        w = torch.matmul(q, k.transpose(-1,-2)) / math.sqrt(k.shape[-1])
        w = torch.softmax(w, dim=-1)
        
        a = torch.matmul(w, v)
        
        a = a.transpose(1,0).reshape(-1, self.vector_size)
        
        a = self.w_0(a)

        return a


In [223]:
#Test unitaire

In [226]:
mha = MultiHeadAttention(2,4,8)
mha(torch.randn(5,8))

tensor([[ 0.0069, -0.1806, -0.4418,  0.5875,  0.2416, -0.4761,  0.3078, -0.4186],
        [ 0.0973, -0.2316, -0.3849,  0.5906,  0.3765, -0.3447,  0.1542, -0.4396],
        [ 0.0887, -0.2287, -0.3749,  0.5695,  0.3461, -0.3592,  0.1308, -0.3994],
        [-0.0118, -0.2070, -0.4307,  0.6249,  0.2939, -0.4722,  0.2506, -0.4576],
        [ 0.0174, -0.1367, -0.4811,  0.5688,  0.1580, -0.5000,  0.4360, -0.3994]],
       grad_fn=<AddmmBackward>)

In [8]:
with open("fr.train.top1M.txt","r",encoding="utf8") as file:
    text = file.read()

a


In [ ]:
tokens = nltk.word_tokenize(text)

In [27]:
LMtransformer = Transformer(Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

criterion = nn.CrossEntropyLoss() #perplexity ?
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.001)

In [18]:
def train_model(nb_epochs, batch_size):
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        for i in range(len(tokens)//batch_size):
            
            batch, y = tokens[i: i+batch_size], tokens[i+batch_size]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            loss = criterion(output, y)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.